# Processing Nanopore data of sams RNAs for machine learning

In [ ]:
# process and mapping Nanopore reads by using Tombo

In [ ]:
# fastq to fasta
awk '(NR - 1) % 4 < 2' path/to/fastq | sed 's/@/>/' > path/to/fasta

In [ ]:
# list up barcoded reads
# sams345_read_id_GUCAUCCC.txt and sams345_read_id_GUCAUGGG.txt will be generated
Rscript extract_barcoded_read_ids.R path/to/fasta

In [ ]:
# subset barcoded reads
fast5_subset --input path/to/fast5 --save_path path/to/fast5_GUCAUCCC --read_id_list sams345_read_id_GUCAUCCC.txt
fast5_subset --input path/to/fast5 --save_path path/to/fast5_GUCAUGGG --read_id_list sams345_read_id_GUCAUGGG.txt

In [ ]:
# unpack fast5 files in case of multifast5
multi_to_single_fast5 -i path/to/fast5_GUCAUCCC -s path/to/fast5_GUCAUCCC_single -t 10
multi_to_single_fast5 -i path/to/fast5_GUCAUGGG -s path/to/fast5_GUCAUGGG_single -t 10

In [ ]:
# Tombo annotating
tombo preprocess annotate_raw_with_fastqs --fast5-basedir path/to/fast5_GUCAUCCC_single --fastq-filenames path/to/fastq --sequencing-summary-filenames path/to/summary.txt --processes 10 --overwrite --corrected-group sams-3_E2E3L_and_sams-5_E2LE3L
tombo preprocess annotate_raw_with_fastqs --fast5-basedir path/to/fast5_GUCAUGGG_single --fastq-filenames path/to/fastq --sequencing-summary-filenames path/to/summary.txt --processes 10 --overwrite --corrected-group sams-3_E2E3L_and_sams-5_E2LE3L

In [ ]:
# Tombo mapping
# use sams-3_E2E3L_and_sams-5_E2LE3L_extension1kb.fa and All_sams_transcripts_extension1kb.fa for in vitro and in vivo, repectively
tombo resquiggle --rna --processes 10 --num-most-common-errors 5 --include-event-stdev --overwrite --corrected-group sams-3_E2E3L_and_sams-5_E2LE3L path/to/fast5_GUCAUCCC_single sams-3_E2E3L_and_sams-5_E2LE3L_extension1kb.fa
tombo resquiggle --rna --processes 10 --num-most-common-errors 5 --include-event-stdev --overwrite --corrected-group sams-3_E2E3L_and_sams-5_E2LE3L path/to/fast5_GUCAUGGG_single sams-3_E2E3L_and_sams-5_E2LE3L_extension1kb.fa

In [ ]:
# plot squiggle
tombo plot genome_locations --fast5-basedirs path/to/fast5_GUCAUGGG_single --control-fast5-basedirs path/to/fast5_GUCAUCCC_single \
--genome-locations sams-3_E2/E3L:1262:+ --overplot-type Downsample --overplot-threshold 100 --num-bases 9 \
--pdf-filename tombo_squiggle_sams-3E2E3L_100reads.pdf